In [12]:
from torch_snippets import *
from PIL import Image
import pandas as pd
import numpy as np
import glob

In [2]:
IMAGE_ROOT = './dataset/images/images'
df_raw = df = pd.read_csv('./dataset/df.csv')

In [10]:
label2target = {l : t + 1 for t,l in enumerate(df_raw['LabelName'].unique())}
label2target['background'] = 0
target2label = {t : l for l,t in label2target.items()}
background_class = label2target['background']
num_classes = len(label2target)
print('target2label : {}'.format(target2label))
print('label2target : {}'.format(label2target))

target2label : {1: 'Bus', 2: 'Truck', 0: 'background'}
label2target : {'Bus': 1, 'Truck': 2, 'background': 0}


In [11]:
def preprocessing_image(img):
    img = torch.tensor(img).permute(2,0,1)
    return img.to(device).float()

In [ ]:
class OpenDataset(torch.utils.data.Dataset):
    w,h = 224, 224
    def __init__(self,df,image_dir = IMAGE_ROOT):
        self.image_dir = image_dir
        self.df = df
        self.files = glob.glob(self.image_dir + '/*')
        self.image_infos = df.ImageID.unique()
    
    def __getitem__(self,ix):
        image_id = self.image_infos[ix]
        img_path = find(image_id,self.files)
        img = Image.open(img_path).convert('RGB')
        img = np.array(img.resize((self.w,self.h), resample=Image.BILINEAR) / 255.)
        data = df[df['ImageID'] == image_id]
        